In [1]:
! cd ../../; git clone https://github.com/alvelvis/ACDC-UD

fatal: destination path 'ACDC-UD' already exists and is not an empty directory.


In [2]:
import sys
sys.path.append('../../ACDC-UD')
import estrutura_ud
import os
import re

# Converter 400 TEM para utf-8
Necessário ter a pasta do Petrolês em utf-16 (no repositório, a conversão para utf-8 já foi realizada, portanto é necessário pular esta etapa)

Etapa necessária, caso contrário o UDPipe terá problemas

In [ ]:
! mkdir processado-utf8
! mkdir gambiarra-utf8
for file in os.listdir("TXT_Teses e Monografias"):
    if file.endswith(".txt"):
        with open(f"TXT_Teses e Monografias/{file}", encoding="utf-16") as f:
            with open(f"processado-utf8/{file}", 'w', encoding="utf-8") as r:
                r.write(f.read())
                
for file in os.listdir("Corpus Gambiarra"):
    if file.endswith(".txt"):
        with open(f"Corpus Gambiarra/{file}", encoding="utf-16") as f:
            with open(f"gambiarra-utf8/{file}", 'w', encoding="utf-8") as r:
                r.write(f.read())             

# Anotar 400TEM
--> DEMORADO

In [3]:
! ../udpipe/udpipe-1.2.0 --tokenize --tag --parse ../udpipe/bosqueud_2.5_workbench.udpipe processado-utf8/raw/*.txt > processado-utf8/udpipe/bem_processado.conllu

Loading UDPipe model: done.
^C


In [5]:
! ../udpipe/udpipe-1.2.0 --tokenize --tag --parse ../udpipe/bosqueud_2.5_workbench.udpipe gambiarra-utf8/raw/*.txt > gambiarra-utf8/udpipe/gambiarra.conllu

Loading UDPipe model: done.
^C


# Adicionar sent_id às frases das 400TEM

In [6]:
! python3 ../scripts/fix_sent_id.py processado-utf8/udpipe/bem_processado.conllu > processado-utf8/udpipe/bem_processado_sentid.conllu

In [7]:
! python3 ../scripts/fix_sent_id.py gambiarra-utf8/udpipe/gambiarra.conllu > gambiarra-utf8/udpipe/gambiarra_sentid.conllu

# Corrigir fórmulas nas 400TEM
(ou só gambiarra)

In [8]:
with open("gambiarra-utf8/udpipe/gambiarra_sentid.conllu") as f:
    gambiarra = f.read()

gambiarra_split = gambiarra.split("\n\n")
print(len(gambiarra_split))
gambiarra = [x for x in gambiarra_split if not "\t\n" in x and not re.search(r'\n[^\d#]', x)]
print(len(gambiarra))

with open("gambiarra-utf8/udpipe/gambiarra_sentid_formula.conllu", 'w') as f:
    f.write("\n\n".join(gambiarra))

1019922
1019916


# Abrir 400TEM UDPIPE
--> DEMORADO

In [9]:
gambiarra = estrutura_ud.Corpus()
gambiarra.load("gambiarra-utf8/udpipe/gambiarra_sentid_formula.conllu")

In [10]:
bemprocessado = estrutura_ud.Corpus()
bemprocessado.load("processado-utf8/udpipe/bem_processado_sentid.conllu")

# Teste de documentos em bemprocessado e gambiarra 
(estranho -- número diferente. Anotar novamente.)

In [11]:
lista_documentos_bemprocessado = []
[lista_documentos_bemprocessado.append(x.rsplit("-", 1)[0]) for x in bemprocessado.sentences if x.rsplit("-", 1)[0] not in lista_documentos_bemprocessado]
lista_documentos_gambiarra = []
[lista_documentos_gambiarra.append(x.rsplit("-", 1)[0]) for x in gambiarra.sentences if x.rsplit("-", 1)[0] not in lista_documentos_gambiarra]

fora_de_gambiarra = [x for x in lista_documentos_bemprocessado if x not in lista_documentos_gambiarra]
fora_de_bemprocessado = [x for x in lista_documentos_gambiarra if x not in lista_documentos_bemprocessado]
print(f"Documentos em bemprocessado ({len(lista_documentos_bemprocessado)}) que faltam em gambiarra ({len(lista_documentos_gambiarra)}):")
print(fora_de_gambiarra)
print("\nDocumentos em gambiarra que faltam em bemprocessado:")
print(fora_de_bemprocessado)
lista_documentos_proibidos = fora_de_gambiarra + fora_de_bemprocessado

Documentos em bemprocessado (403) que faltam em gambiarra (415):
['343-20150401-TESEMSC_0']

Documentos em gambiarra que faltam em bemprocessado:
['1-20140905-TESEDSC_0', '140-20141209-TESEMSC_0', '18-20140905-TESEDSC_0', '255-20150506-MONOGRAFIA_0', '273-20140930-MONOGRAFIA_0', '29-20150519-TESEDSC_0', '37-20150226-TESEDSC_0', '43-20150519-MONOGRAFIA_0', '5-20140905-TESEDSC_0', '78-20140905-TESEDSC_0', '79-20140905-TESEDSC_0', '89-20141112-TESEDSC_0', '9-20140905-TESEDSC_0']


## Normalização dos documentos em ambos os arquivos

In [12]:
print(f"Documentos proibidos {(len(lista_documentos_proibidos))}:")
print(lista_documentos_proibidos)
gambiarra.sentences = {x: y for x, y in gambiarra.sentences.items() if x.rsplit("-", 1)[0] not in lista_documentos_proibidos}
bemprocessado.sentences = {x: y for x, y in bemprocessado.sentences.items() if x.rsplit("-", 1)[0] not in lista_documentos_proibidos}

lista_documentos_bemprocessado = []
[lista_documentos_bemprocessado.append(x.rsplit("-", 1)[0]) for x in bemprocessado.sentences if x.rsplit("-", 1)[0] not in lista_documentos_bemprocessado]
lista_documentos_gambiarra = []
[lista_documentos_gambiarra.append(x.rsplit("-", 1)[0]) for x in gambiarra.sentences if x.rsplit("-", 1)[0] not in lista_documentos_gambiarra]

fora_de_gambiarra = [x for x in lista_documentos_bemprocessado if x not in lista_documentos_gambiarra]
fora_de_bemprocessado = [x for x in lista_documentos_gambiarra if x not in lista_documentos_bemprocessado]
print(f"\nDocumentos em bemprocessado ({len(lista_documentos_bemprocessado)}) que faltam em gambiarra ({len(lista_documentos_gambiarra)}):")
print(fora_de_gambiarra)
print("\nDocumentos em gambiarra que faltam em bemprocessado:")
print(fora_de_bemprocessado)

Documentos proibidos:
['343-20150401-TESEMSC_0', '1-20140905-TESEDSC_0', '140-20141209-TESEMSC_0', '18-20140905-TESEDSC_0', '255-20150506-MONOGRAFIA_0', '273-20140930-MONOGRAFIA_0', '29-20150519-TESEDSC_0', '37-20150226-TESEDSC_0', '43-20150519-MONOGRAFIA_0', '5-20140905-TESEDSC_0', '78-20140905-TESEDSC_0', '79-20140905-TESEDSC_0', '89-20141112-TESEDSC_0', '9-20140905-TESEDSC_0']

Documentos em bemprocessado (402) que faltam em gambiarra (402):
[]

Documentos em gambiarra que faltam em bemprocessado:
[]


In [13]:
gambiarra.save('gambiarra-utf8/udpipe/gambiarra_sentid_formula_normalizado.conllu')
bemprocessado.save('processado-utf8/udpipe/bemprocessado_sentid_normalizado.conllu')

# Anotar com jPTDP
--> DEMORADO

In [15]:
! sudo -S apt install python-pip --yes < ~/sudopass
! pip install cython numpy
! pip install dynet==2.0.3
! cd ../jPTDP; wget https://www.dropbox.com/s/fn0r48xhn67inpt/outputs.tar.gz; tar -xvzf outputs.tar.gz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.1).
The following package was automatically installed and is no longer required:
  libdumbnet1
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
  Using cached https://files.pythonhosted.org/packages/7b/d2/060a4f311c3b4a83cb050882f1032dabd5c8045b489dc699cff60bcebdba/Cython-0.29.14-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3a/5f/47e578b3ae79e2624e205445ab77a1848acdaa2929a00eeef6b16eaaeb20/numpy-1.16.6-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/4f/de/181a8380e9fdb89d9aa5838059336bb535503d5f2053e621438e69081407/dyNET-2.0.3-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3a/5f/47e578b3ae79e2624e205445ab77a1848acdaa2929a00eeef6b16eaaeb20/numpy-1.16

In [ ]:
! python ../jPTDP/src/jPTDP.py --predict --model ../jPTDP/outputs/jPTDP_pt_ud.model --params ../jPTDP/outputs/jPTDP_pt_ud.params --test gambiarra-utf8/udpipe/gambiarra_sentid_formula_normalizado.conllu --outdir gambiarra-utf8/jptdp --output gambiarra_jptdp.conllu --dynet-mem 16000

[dynet] random seed: 3378766694
[dynet] allocating memory: 8000MB
[dynet] memory allocation done.
Loading pre-trained model
Predicting POS tags and parsing dependencies


In [ ]:
! python ../jPTDP/src/jPTDP.py --predict --model ../jPTDP/outputs/jPTDP_pt_ud.model --params ../jPTDP/outputs/jPTDP_pt_ud.params --test gambiarra-utf8/udpipe/gambiarra_sentid_formula_normalizado_4_fevereiro.conllu --outdir gambiarra-utf8/jptdp --output gambiarra_jptdp_4_fevereiro.conllu --dynet-mem 16000

[dynet] random seed: 2794690350
[dynet] allocating memory: 16000MB
[dynet] memory allocation done.
Loading pre-trained model
Predicting POS tags and parsing dependencies


In [ ]:
! python ../jPTDP/src/jPTDP.py --predict --model ../jPTDP/outputs/jPTDP_pt_ud.model --params ../jPTDP/outputs/jPTDP_pt_ud.params --test processado-utf8/udpipe/bemprocessado_sentid_normalizado.conllu --outdir processado-utf8/jptdp --output bemprocessado_jptdp.conllu --dynet-mem 16000

[dynet] random seed: 1083583026
[dynet] allocating memory: 16000MB
[dynet] memory allocation done.
Loading pre-trained model
Predicting POS tags and parsing dependencies


# Abrir 400TEM jPTDP
--> DEMORADO

In [ ]:
gambiarra = estrutura_ud.Corpus()
gambiarra.load("gambiarra-utf8/jptdp/gambiarra_jptdp.conllu")

In [ ]:
bemprocessado = estrutura_ud.Corpus()
bemprocessado.load("processado-utf8/jptdp/bemprocessado_jptdp.conllu")

# Volte três casas: teste e normalização em jPTDP

# Distribuição de DP e POS (ajustar onde)

In [ ]:
dic_pos_dp_gambiarra = {}
dic_pos_dp_bemprocessado = {}
for sent, sentence in gambiarra.sentences.items():
    for t, token in enumerate(sentence.tokens):
        if not '-' in token.id:
            dic_pos_dp_gambiarra[token.upos] = 1 if not token.upos in dic_pos_dp_gambiarra else dic_pos_dp_gambiarra[token.upos] + 1
            dic_pos_dp_gambiarra[token.deprel] = 1 if not token.deprel in dic_pos_dp_gambiarra else dic_pos_dp_gambiarra[token.deprel] + 1
for sent, sentence in bemprocessado.sentences.items():
    for t, token in enumerate(sentence.tokens):
        if not '-' in token.id:
            dic_pos_dp_bemprocessado[token.upos] = 1 if not token.upos in dic_pos_dp_bemprocessado else dic_pos_dp_bemprocessado[token.upos] + 1
            dic_pos_dp_bemprocessado[token.deprel] = 1 if not token.deprel in dic_pos_dp_bemprocessado else dic_pos_dp_bemprocessado[token.deprel] + 1

print(f"Classes em gambiarra ({len(dic_pos_dp_gambiarra)})")
print(sorted(dic_pos_dp_gambiarra))
print(f"\nClasses em bemprocessado ({len(dic_pos_dp_bemprocessado)})")
print(sorted(dic_pos_dp_bemprocessado))

# Número de sentenças e tokens

In [ ]:
documentos = []
[documentos.append(x.rsplit("-", 1)[0]) for x in bemprocessado.sentences if x.rsplit("-", 1)[0] not in documentos]
print(f"{len(documentos)} documentos foram encontrados em FINAL")
print("\n".join(["* " + x for x in documentos]))

# a contagem de tokens não leva em consideração aqueles que têm "-" no seu id (indicação de contração (a contração em si conta, é claro))
print("\n|Versão|Sentenças|Tokens|")
print("|---|---|---|")
tokens_gambiarra = len([x for sentence in gambiarra.sentences.values() for x in sentence.tokens if not '-' in x.id])
tokens_final = len([x for sentence in bemprocessado.sentences.values() for x in sentence.tokens if not '-' in x.id])
print("|GAMBIARRA|{}|{}|".format(len(gambiarra.sentences), tokens_gambiarra))
print("|FINAL|{}|{}|".format(len(bemprocessado.sentences), tokens_final))

# Distribuição das categorias bruta

In [ ]:
print("\nDistribuição das categorias bruta")
print("| {:25} | {:25} | {:25} |".format('Anotação', 'GAMBIARRA', 'FINAL'))
print("| {:25} | {:25} | {:25} |".format("---", "---", "---"))
lista_pos_dp = [x for x in dic_pos_dp_gambiarra]
[lista_pos_dp.append(x) for x in dic_pos_dp_bemprocessado if not x in lista_pos_dp]
for pos_dp in sorted(lista_pos_dp):
    print("| {:25} | {:25} | {:25} |".format(pos_dp, dic_pos_dp_gambiarra[pos_dp] if pos_dp in dic_pos_dp_gambiarra else 0, dic_pos_dp_bemprocessado[pos_dp] if pos_dp in dic_pos_dp_bemprocessado else 0))

# Distribuição das categorias relativa ao total de tokens da versão

In [ ]:
print("\nDistribuição das categorias relativa ao total de tokens na versão")
print("| {:25} | {:25} | {:25} |".format('Anotação', 'GAMBIARRA', 'FINAL'))
print("| {:25} | {:25} | {:25} |".format("---", "---", "---"))
lista_pos_dp = [x for x in dic_pos_dp_gambiarra]
[lista_pos_dp.append(x) for x in dic_pos_dp_bemprocessado if not x in lista_pos_dp]
for pos_dp in sorted(lista_pos_dp):
    print("| {:25} | {:25} | {:25} |".format(pos_dp, str((dic_pos_dp_gambiarra[pos_dp]/tokens_gambiarra)*100)+'%' if pos_dp in dic_pos_dp_gambiarra else 0, str((dic_pos_dp_bemprocessado[pos_dp]/tokens_final)*100)+'%' if pos_dp in dic_pos_dp_bemprocessado else 0))

# Visualização dos lemas

In [ ]:
lista_lemas_gambiarra = [lista_lemas_gambiarra.append(x.lemma) for y in gambiarra.sentences.values() for x in y.tokens if not '-' in x.id]
lista_lemas_bemprocessado = [lista_lemas_bemprocessado.append(x.lemma) for y in bemprocessado.sentences.values() for x in y.tokens if not '-' in x.id]

print("|{:25}|{:25}|{:25}|".format('', 'GAMBIARRA', 'FINAL'))
print("|{:25}|{:25}|{:25}|".format("---", "---", "---"))
print("|{:25}|{:25}|{:25}|".format('Lemas diferentes', len(list(dict.fromkeys(lista_lemas_gambiarra))), len(list(dict.fromkeys(lista_lemas_bemprocessado)))))

In [ ]:
print("\nDistribuição dos lemas em GAMBIARRA")
print("|{:25}|{:25}|{:25}|".format('LEMA', 'OCORRÊNCIA', 'REPRESENTATIVIDADE'))
#print("|{:25}|{:25}|{:25}|".format('', 'GAMBIARRA', 'FINAL'))
[
    print("|{:25}|{:25}|{:25}|".format(
            x, 
            lista_lemas_gambiarra.count(x), 
            str((lista_lemas_gambiarra.count(x)/len(lista_lemas_gambiarra))*100)+'%',
        ))
        for x in sorted(lista_lemas_gambiarra, reverse=True, key=lambda y: lista_lemas_gabiarra.count(y))
]